 # 第7章: データベース
 artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．

 |フィールド|型|内容|例|
 |---|---|---|---|
 |id|ユニーク識別子|整数|20660|
 |gid|グローバル識別子|文字列|"ecf9f3a3-35e9-4c58-acaa-e707fba45060"|
 |name|アーティスト名|文字列|"Oasis"|
 |sort_name|アーティスト名（辞書順整列用）|文字列|"Oasis"|
 |area|活動場所|文字列|"United Kingdom"|
 |aliases|別名|辞書オブジェクトのリスト||
 |aliases[].name|別名|文字列|"オアシス"|
 |aliases[].sort_name|別名（整列用）|文字列|"オアシス"|
 |begin|活動開始日|辞書||
 |begin.year|活動開始年|整数|1991|
 |begin.month|活動開始月|整数||
 |begin.date|活動開始日|整数||
 |end|活動終了日|辞書||
 |end.year|活動終了年|整数|2009|
 |end.month|活動終了月|整数|8|
 |end.date|活動終了日|整数|28|
 |tags|タグ|辞書オブジェクトのリスト||
 |tags[].count|タグ付けされた回数|整数|1|
 |tags[].value|タグ内容|文字列|"rock"|
 |rating|レーティング|辞書オブジェクト||
 |rating.count|レーティングの投票数|整数|13|
 |rating.value|レーティングの値（平均値）|整数|86|

----

 ## 60. KVSの構築
 Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

 ### 手順
 * KVSとして`Redis`を使います
 * `$ brew install redis`
 * `$ brew services start redis`
 * `$ redis-cli`で起動されているかどうかとポート番号がわかる
 * pythonのライブラリは`redis-py`を使う
 * `$ pip install redis`
 * `name => area`のデータベースを構築する
 * nameが重複しているデータが複数ある？
   * nameは一意だと思っていたが、同じアーティスト名で活動してる別なアーティストがいるらしい
     * 例: Queen
   * idでプレフィックスをつけることにした

In [2]:
import gzip
import json
import redis
r = redis.Redis(host='localhost', port=6379, db=0)

with gzip.open("artist.json.gz", "rt") as f:
  for line in f:
    json_dict = json.loads(line)
    if 'name' in json_dict:
      r.set(f"{json_dict['name']}:{json_dict.get('id', 0)}", json_dict.get('area', ''))

----

 ## 61. KVSの検索
 60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．
 ### メモ
 * `$ redis-cli`
 * `$ get key`で対応したvalueが返ってくる
 * `$ keys word`でキーを高速に検索することができる
 * ~~`keys name:*`して出力されたkeyに対して`get key`する~~
 * `$ mget keys`でkeyを複数指定して一気に取ってこれるらしいのでこれを使う

In [3]:
import redis

def get_area(name):
  r = redis.Redis(host='localhost', port=6379, db=0)
  areas = r.mget(r.keys(f'{name}:*'))
  return [x for x in set(areas) if x]

In [4]:
get_area('The Silhouettes')

[b'United States', b'Netherlands']

In [5]:
get_area('Al Street')

[b'United States']

----

 ## 62. KVS内の反復処理
 60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．
 ### メモ
 * valueを検索するコマンドは無い => KVS内の反復処理
 * `$ keys *`で全件取得してvalueが`Japan`かどうかを判別するしかない
 * めちゃくちゃ時間がかかる

In [7]:
import redis
r = redis.Redis(host='localhost', port=6379, db=0)

i = 0
for key in r.keys():
  if r.get(key) == b'Japan':
    i += 1

i

22821

----

 ## 63. オブジェクトを値に格納したKVS
 KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [9]:
import gzip
import json
import redis
r = redis.Redis(host='localhost', port=6379, db=1)

with gzip.open("artist.json.gz", "rt") as f:
  for (i, data) in enumerate(f):
    json_dict = json.loads(data)
    if i % 100000 == 0:
      print(i)
    if 'name' in json_dict:
      tags = json_dict.get('tags', [{'count': 0, 'value': ''}])
      count = 0
      for tag in tags:
        count += tag['count']
      value = {
        'tags': str([val['value'] for val in tags]),
        'tagsCount': count
      }
      r.hmset(f"{json_dict['name']}:{json_dict.get('id', 0)}", value)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000


In [10]:
def get_tags(name):
  r = redis.Redis(host='localhost', port=6379, db=1)
  keys = r.keys(f'{name}:*')
  res = []
  for key in keys:
    tags = r.hmget(key, 'tags', 'tagsCount')
    if int(tags[1]) == 0:
      continue
    res.append({'tags': tags[0], 'count': int(tags[1])})
  return res

get_tags('Queen')

[{'tags': b"['kamen rider w', 'related-akb48']", 'count': 2},
 {'tags': b"['hard rock', '70s', 'queen family', '90s', '80s', 'glam rock', 'british', 'english', 'uk', 'pop/rock', 'pop-rock', 'britannique', 'classic pop and rock', 'queen', 'united kingdom', 'langham 1 studio bbc', 'kind of magic', 'band', 'rock', 'platinum']",
  'count': 30}]

----

 ## 64. MongoDBの構築
 アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value
 ### メモ
 * `$ brew install mongodb`
 * `$ brew services start mongodb`
 * connecting to: mongodb://127.0.0.1:27017
 * `pymongo`を使います
 * `$ pip install pymongo`

In [11]:
import gzip
import json
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.nlp100
col = db.artist

# 存在した場合、初期化
col.drop()

with gzip.open("artist.json.gz", "rt") as f:
  batch = []
  for (i, data) in enumerate(f):
    # パフォーマンスを考えて、10000件ごとに格納していく
    json_dict = json.loads(data)
    batch.append(json_dict)
    if i % 10000 == 0:
      col.insert_many(batch)
      print(i)
      batch = []
    
  if len(batch) > 0:
    col.insert_many(batch)
    print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
921336


In [12]:
from pymongo import IndexModel, ASCENDING
col.create_indexes([
  IndexModel([('name', ASCENDING)]),
  IndexModel([('aliases.name', ASCENDING)]),
  IndexModel([('tags.value', ASCENDING)]),
  IndexModel([('rating.value', ASCENDING)])
])

['name_1', 'aliases.name_1', 'tags.value_1', 'rating.value_1']

In [13]:
col.find_one()

{'_id': ObjectId('5ce23f73506bf4ffeeb67837'),
 'name': 'WIK▲N',
 'tags': [{'count': 1, 'value': 'sillyname'}],
 'sort_name': 'WIK▲N',
 'ended': True,
 'gid': '8972b1c1-6482-4750-b51f-596d2edea8b1',
 'id': 805192}

----

 ## 65. MongoDBの検索
 MongoDBのインタラクティブシェルを用いて，`"Queen"`というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．
 ## 操作
 * `$ mongo`
 * `> use nlp100`
 * `> db.artist.find({"name": "Queen"})`

In [14]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.nlp100

find = db.artist.find({'name': 'Queen'})
[f for f in find]

[{'_id': ObjectId('5ce23fa8506bf4ffeebfaf5b'),
  'name': 'Queen',
  'area': 'Japan',
  'gender': 'Female',
  'tags': [{'count': 1, 'value': 'kamen rider w'},
   {'count': 1, 'value': 'related-akb48'}],
  'sort_name': 'Queen',
  'ended': True,
  'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
  'type': 'Character',
  'id': 701492,
  'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]},
 {'_id': ObjectId('5ce23fab506bf4ffeec07607'),
  'rating': {'count': 24, 'value': 92},
  'begin': {'date': 27, 'month': 6, 'year': 1970},
  'name': 'Queen',
  'area': 'United Kingdom',
  'tags': [{'count': 2, 'value': 'hard rock'},
   {'count': 1, 'value': '70s'},
   {'count': 1, 'value': 'queen family'},
   {'count': 1, 'value': '90s'},
   {'count': 1, 'value': '80s'},
   {'count': 1, 'value': 'glam rock'},
   {'count': 4, 'value': 'british'},
   {'count': 1, 'value': 'english'},
   {'count': 2, 'value': 'uk'},
   {'count': 1, 'value': 'pop/rock'},
   {'count': 1, 'value': 'pop-rock'},
   {'count': 1, 'v

----

 ## 66. 検索件数の取得
 MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ．
 ## 操作
 * `$ mongo`
 * `> use nlp100`
 * `> db.artist.find({"area": "Japan"}).count()`
 * `> db.artist.find({"area": "Japan"}).length()`でもいけた
 * `> db.artist.find({"area": "Japan"}).size()`でもいけた

In [15]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.nlp100

db.artist.count_documents({'area': 'Japan'})

22821

----

 ## 67. 複数のドキュメントの取得
 特定の（指定した）別名を持つアーティストを検索せよ．

In [16]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.nlp100

def get_artist_aliases(name):
  find = db.artist.find({'aliases.name': name})
  return list(find)

In [17]:
get_artist_aliases('Queen')

[{'_id': ObjectId('5ce23fa8506bf4ffeebfaf5b'),
  'name': 'Queen',
  'area': 'Japan',
  'gender': 'Female',
  'tags': [{'count': 1, 'value': 'kamen rider w'},
   {'count': 1, 'value': 'related-akb48'}],
  'sort_name': 'Queen',
  'ended': True,
  'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
  'type': 'Character',
  'id': 701492,
  'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]}]

----

 ## 68. ソート
 "dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．
 ## メモ
 * 投票数は`rating.count`
 * `> db.artist.find({"tags.value": "dance"}).sort({"rating.count":-1}).limit(10)

In [18]:
from pymongo import MongoClient, DESCENDING
client = MongoClient('localhost', 27017)
db = client.nlp100

res = db.artist.find({'tags.value': 'dance'}).sort('rating.count', DESCENDING).limit(10)

[(artist['name'], artist['rating']) for artist in res]

[('Madonna', {'count': 26, 'value': 88}),
 ('Björk', {'count': 23, 'value': 84}),
 ('The Prodigy', {'count': 23, 'value': 90}),
 ('Rihanna', {'count': 15, 'value': 68}),
 ('Britney Spears', {'count': 13, 'value': 83}),
 ('Maroon 5', {'count': 11, 'value': 60}),
 ('Adam Lambert', {'count': 7, 'value': 100}),
 ('Fatboy Slim', {'count': 7, 'value': 77}),
 ('Basement Jaxx', {'count': 6, 'value': 83}),
 ('Cornershop', {'count': 5, 'value': 68})]

----

 ## 69. Webアプリケーションの作成
 ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．
 ## サーバー側
 * `Flask`というPythonのwebフレームワークを使う
 ### 環境構築
 * `$ pip install flask`
 ## クライアント側
 * `Vue.js`というJavaScriptフレームワークを使う
 * `templete`・`script`・`style`の3つに分けてHTMLとJavaScriptのコードを記述することができる
 * DOMの構造を直接いじることなく、リアクティブな変更を行うことができる
 ### 環境構築
 * Vue.jsのプロジェクトを作る時は`vue cli`というツールを使うのがオススメ
 * 使うにはNode.jsをインストールするときに付属してくる`npm`(Pythonでいうところの`pip`)が必要
 * Nodeのインストール方法は省略
 * `$ npm install -g @vue/cli`
 * `$ vue create nlp100-69`
 * これでVue.jsのプロジェクトの雛形が作成される
 * サーバーとのやりとりのために`axios`というライブラリを使う
 * `$ npm install axios`